In [ ]:
import re
import zlib
import base64
import datetime
import numpy as np
import pandas as pd
import json
import configparser
from utils import *
from sys import platform
from elasticsearch import Elasticsearch

# import torch
# from tslearn.metrics import dtw, dtw_path
# from tslearn.metrics import lcss, lcss_path
# from transformers import BertTokenizer, BertModel
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# tokenizer = BertTokenizer.from_pretrained('google/bert_uncased_L-2_H-128_A-2')
# model = BertModel.from_pretrained("google/bert_uncased_L-2_H-128_A-2")
# model = model.to(device)
# device

In [ ]:
cf = configparser.ConfigParser()
cf.read('config/config.cfg')

env = 'DEVELOP'
if 'win' in platform:
    env = 'DEVELOP'
elif 'linux' in platform:
    env = 'PRODUCT'
    
class EsCtrl(object):
    def __init__(self):
        self.es_ctrl = Elasticsearch(cf['ENV_'+env]['ADDR'], ca_certs=cf['ELASTICSEARCH']['CA_CERTS'])

    def query_index_logs(self, index):
        # query = {
        #     "match": {
        #         "trace": "com_ericsson_trithread:INFO"
        #     }
        # }
        #data = self.es_ctrl.search(index=index, query=query, scroll='1s', size=10000)
        data = self.es_ctrl.search(index=index, scroll='1s', size=10000)
        sid = data['_scroll_id']
        scroll_size = len(data['hits']['hits'])
        res = []
        while scroll_size > 0:
            # Before scroll, process current batch of hits
            res.extend(data['hits']['hits'])
            data = self.es_ctrl.scroll(scroll_id=sid, scroll='1s')
            # Update the scroll ID
            sid = data['_scroll_id']
            # Get the number of results that returned in the last scroll
            scroll_size = len(data['hits']['hits'])
        return res

    def query_indices(self):
        res = []
        for key in self.es_ctrl.indices.get_alias().keys():
            if len(key) > 0:
                if '.analyzed_' in key:
                    res.append(key.replace('.analyzed_', ''))
        return res

    def is_exists(self, index):
        return self.es_ctrl.indices.exists(index=index)

    def count_index(self, index):
        return self.es_ctrl.count(index=index)['count']

    def store_index(self, index, data):
        data = deflate_and_base64_encode(json.dumps(data).encode('utf-8'))
        return self.es_ctrl.index(index=index, body={'content': data})

    def query_index(self, index):
        data = self.es_ctrl.search(index=index)
        data = json.loads(decode_base64_and_inflate(data['hits']['hits'][0]['_source']['content']))
        return data

In [ ]:
############################################ Data Clean ################################################
def package_kv(df):
    res = {}
    k_type = {}
    for i, (kv,index,timestamp) in enumerate(zip(df.kv.values, df['index'].values, df.timestamp.values)):
        for item in kv:
            if len(item[1]) > 0:
                if item[0] in res:
                    res[item[0]].append(item[1]  + [timestamp] + [str(i)] + [index]) # [value1,value2,value3,timestamp,process_index,global_index]
                else:
                    res[item[0]] = [item[1]  + [timestamp] + [str(i)] + [index]]
    for key in res.keys():
        width = max(map(len, res[key])) # get max width
        type_list = []
        for i, item in enumerate(res[key]):
            if '0x' in item[0]:
                type_list.append('register')
            elif item[0].isupper():
                type_list.append('discrete')
            else:
                type_list.append('continuous')
                
            if len(item) != width:
                tmp = [0 for _ in range(0, width)]
                tmp[-3] = item[-3]
                tmp[-2] = item[-2]
                tmp[-1] = item[-1]
                res[key][i] = tmp
        res[key] = np.array(res[key]).transpose().tolist() # matrix transposition
        k_type[key] = 'discrete' if len(set(type_list)) > 1 else list(set(type_list))[0]
    return res,k_type


def package_inverted_index_table(table, key, data):
    def clean_special_symbols(text):
        for ch in ['/','*','{','}','[',']','(',')','#','+','-','!','=',':',',','"']:
            if ch in text:
                text = text.replace(ch," ")
        return re.sub(" +", " ", text)
    for index, msg in data:
        for word in set(clean_special_symbols(msg).split(' ')):
            w = word.lower()
            if w not in table:
                table[w] = {'x': [index], 'y': [key]} # x:global index, y: yaxis num 
            else:
                table[w]['x'].append(index)
                table[w]['y'].append(key)


def clean_data(esdata):
    def clean_msg_special_symbols(text):
        for ch in ['{', '}', '[', ']', '(', ')', '"', '::']:
            if ch in text:
                text = text.replace(ch, " ")
        return re.sub(" +", " ", text)

    story = []
    for item in esdata:
        if 'msg' in item['_source']:
            tmp = clean_msg_special_symbols(item['_source']['msg'])
            if len(re.findall('process \= (.*?)$', tmp)) > 0:
                process = re.findall('process \= (.*?),', tmp)[0]
                msg = re.findall('msg \= (.*?)$', tmp)[0]
            #                 fileAndLine = re.findall('fileAndLine \= \"(.*?)\"',item['_source']['msg'])[0].split(':')[0]
            # elif len(re.findall('procname \= (.*?)$', tmp)) > 0:
            #     process = re.findall('procname \= (.*?),', tmp)[0]
            #     #                 msg = tmp.split(',')[2].replace('"','').replace('}','').replace('{','')
            #     msg = tmp
            else:
                process = 'main'
                msg = tmp
                
            msg = msg.replace('= ',':').replace(' = ',':').replace(': ',':').replace(' : ',':').replace('=',':')

            for elm in re.split('[: ]',msg):
                if elm.isupper():
                    msg = re.sub('[: ]'+elm, ':'+elm, msg)

            msg = re.sub('(:(?!-).*?[ $])', r'\1,', (msg + ' $'))
            # msg
            kv = []
            for k, v in re.findall('([A-Za-z0-9_.]+?)[ ]?[:=][ ]?(.*?)[,$]', msg):
                if len(v.strip()) > 0:
                    if (v.strip()+'xx').lower()[0:2] == '0x':
                        kv.append((k.strip()+'(r)',  [v.strip()]))
                    elif v.strip()[0].isalpha():
                        kv.append((k.strip()+'(d)', [v.strip()]))
                    else:
                        kv.append((k.strip()+'(c)', re.findall('[0-9.]+', v)))
                        
            millisecond = str(item['_source']['millisecond'])
            supply_zero = ''
            for _ in range(0, 9-len(millisecond)):
                supply_zero = supply_zero + '0'
            millisecond = supply_zero + millisecond
            story.append([item['_source']['device'], item['_source']['trace'], process,  item['_source']['logtime'][:-1] + '.' + millisecond, item['_source']['msg'], kv])
            
    story = pd.DataFrame(story, columns=['device', 'trace', 'process', 'timestamp', 'msg', 'kv']).sort_values('timestamp',ascending=True).reset_index(drop=True)

    story_line = {}
    inverted_index_table = {}
    for dev in set(story.device.values):
        data = story.loc[(story['device'] == dev), :].reset_index(drop=True)
        sub_inverted_index_table = {}
        for i, process_name in enumerate(sorted(set(data.process.values), key=list(data.process.values).index)):
            process = data.loc[(data['process'] == process_name), :].reset_index()
            process['index'] = process['index'].astype(str)
            process_start_time = process['timestamp'][0]
            process_start_count = process['index'][0]
            process_end_time = process['timestamp'][process.shape[0] - 1]
            process_end_count = process['index'][process.shape[0] - 1]
            package_inverted_index_table(sub_inverted_index_table, i, zip(process['index'].values, process.msg.values))
            msg = dict(zip(process['index'].values, [str(a) + '||' + b + '||' + c for a, b, c in
                                                     zip(process.index.values, process.timestamp.values,
                                                         process.msg.values)]))

            kv,k_type = package_kv(process)
            if dev not in story_line:
                story_line[dev] = [{'process': process_name, 'start_time': process_start_time, 'start_count': process_start_count, 'end_time': process_end_time, 'end_count': process_end_count, 'msg': msg, 'kv': kv}]
            else:
                story_line[dev].append({'process': process_name, 'start_time': process_start_time, 'start_count': process_start_count, 'end_time': process_end_time, 'end_count': process_end_count, 'msg': msg, 'kv': kv})
        inverted_index_table[dev] = sub_inverted_index_table
    return {'story_line': story_line, 'inverted_index_table': inverted_index_table}


def apply_filter_by_keywords(df):
    if (len(set(df['msg']) & set([':', '='])) > 0):
        return False
    return True


def apply_filter_digit(df):
    return re.sub('\d+', '', df['msg'])


def apply_keyword_highlight(df, keywords, color_highlight):
    tmp = [item.lower() for item in keywords]
    for item in tmp:
        if (item == 'abn:') & (item in df['msg'].lower()):
            return color_highlight
        elif len(set(df['msg'].lower().split(' ')).intersection(set(tmp))) > 0:
            return color_highlight
    return df['status']


def cal_time_difference(start, end):
    return datetime.datetime.strptime(end, "%H:%M:%S") - datetime.datetime.strptime(start, "%H:%M:%S")


############################################ XML Compression and Decompression ################################################
def decode_base64_and_inflate(b64string):
    decoded_data = base64.b64decode(b64string)
    return zlib.decompress(decoded_data , -15)


def deflate_and_base64_encode(string_val):
    zlibbed_str = zlib.compress(string_val)
    compressed_string = zlibbed_str[2:-4]
    return base64.b64encode(compressed_string).decode("utf-8")

############################################ Text ecoder ################################################
def pretrained_model_encode_msg(object1, object2):
    with torch.no_grad():
        object1_inputs = tokenizer(list(object1.msg.values), padding=True, truncation=True, return_tensors="pt").to(device)
        object1_outputs = model(**object1_inputs)

        object2_inputs = tokenizer(list(object2.msg.values), padding=True, truncation=True, return_tensors="pt").to(device)
        object2_outputs = model(**object2_inputs)
    return object1_outputs, object2_outputs

def onehot_encode_string(str1,str2):
    data = []
    data.extend(str1)
    data.extend(str2)
    values = np.array(data)

    # integer encode
    label_encoder = LabelEncoder().fit(values)
    integer_encoded = label_encoder.transform(values)

    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoder = OneHotEncoder(sparse=False).fit(integer_encoded)

    encoder1 = onehot_encoder.transform(label_encoder.transform(str1).reshape(len(str1), 1))
    encoder2 = onehot_encoder.transform(label_encoder.transform(str2).reshape(len(str2), 1))
    # inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
    # print(inverted)
    return encoder1,encoder2

############################################ Space Vectors Algorithm ################################################
def cal_lcss_path_and_score(s_y1, s_y2):
    path, score = lcss_path(s_y1, s_y2)
    return path, score

def cal_dtw_path_and_score(s_y1, s_y2):
    path, score = dtw_path(s_y1, s_y2)
    return path, score

In [ ]:
from extract import *

with open(cf['ENV_'+env]['LOG_STORE_PATH'] + 'ru_lock_unlock_dpd_hw_fault_air6419_mongoose_2022_10_10', "rb") as myfile:
    S = myfile.read()
res = json.loads(gzip.decompress(S))

In [ ]:
import numpy
import matplotlib.pyplot as plt

from tslearn.generators import random_walks
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn import metrics


numpy.random.seed(0)
n_ts, sz, d = 2, 100, 1
dataset = random_walks(n_ts=n_ts, sz=sz, d=d, random_state=5)
scaler = TimeSeriesScalerMeanVariance(mu=0., std=1.)  # Rescale time series
dataset_scaled = scaler.fit_transform(dataset)

lcss_path, sim_lcss = metrics.lcss_path(dataset_scaled[0, :, 0], dataset_scaled[1, :40, 0], eps=1.5)
dtw_path, sim_dtw = metrics.dtw_path(dataset_scaled[0, :, 0], dataset_scaled[1, :40, 0])

plt.figure(1, figsize=(8, 8))

plt.plot(dataset_scaled[0, :, 0], "b-", label='First time series')
plt.plot(dataset_scaled[1, :40, 0], "g-", label='Second time series')

for positions in lcss_path:
    plt.plot([positions[0], positions[1]],
             [dataset_scaled[0, positions[0], 0], dataset_scaled[1, positions[1], 0]], color='orange')
plt.legend()
plt.title("Time series matching with LCSS")

plt.figure(2, figsize=(8, 8))
plt.plot(dataset_scaled[0, :, 0], "b-", label='First time series')
plt.plot(dataset_scaled[1, :40, 0], "g-", label='Second time series')

for positions in dtw_path:
    plt.plot([positions[0], positions[1]],
             [dataset_scaled[0, positions[0], 0], dataset_scaled[1, positions[1], 0]], color='orange')

plt.legend()
plt.title("Time series matching with DTW")

plt.tight_layout()
plt.show()

In [ ]:
kv_a = [[int(item)] for item in story_a['kv']['txlProcBranchH']['txAtt(c)'][0]]
kv_b = [[int(item)] for item in story_b['kv']['txlProcBranchH']['txAtt(c)'][0]]
dataset = [kv_a, kv_b]
dataset

In [ ]:
import numpy
import matplotlib.pyplot as plt

from tslearn.generators import random_walks
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn import metrics


# numpy.random.seed(0)
# n_ts, sz, d = 2, 100, 1
# dataset = random_walks(n_ts=n_ts, sz=sz, d=d, random_state=5)

kv_a = [int(item) for item in story_a['kv']['txlProcBranchH']['txAtt(c)'][0]]
kv_b = [int(item) for item in story_b['kv']['txlProcBranchH']['txAtt(c)'][0]]
[kv_b.insert(0,0) for _ in range(0, len(kv_a) - len(kv_b))]

dataset = [kv_a, kv_b]
scaler = TimeSeriesScalerMeanVariance(mu=0., std=1.)  # Rescale time series
dataset_scaled = scaler.fit_transform(dataset)

lcss_path, sim_lcss = metrics.lcss_path(dataset_scaled[0, :, 0], dataset_scaled[1, :, 0], eps=1.5)
dtw_path, sim_dtw = metrics.dtw_path(dataset_scaled[0, :, 0], dataset_scaled[1, :, 0])

plt.figure(1, figsize=(8, 8))

plt.plot(dataset_scaled[0, :, 0], "b-", label='First time series')
plt.plot(dataset_scaled[1, :, 0], "g-", label='Second time series')

for positions in lcss_path:
    plt.plot([positions[0], positions[1]],
             [dataset_scaled[0, positions[0], 0], dataset_scaled[1, positions[1], 0]], color='orange')
plt.legend()
plt.title("Time series matching with LCSS")

plt.figure(2, figsize=(8, 8))
plt.plot(dataset_scaled[0, :, 0], "b-", label='First time series')
plt.plot(dataset_scaled[1, :, 0], "g-", label='Second time series')

for positions in dtw_path:
    plt.plot([positions[0], positions[1]],
             [dataset_scaled[0, positions[0], 0], dataset_scaled[1, positions[1], 0]], color='orange')

plt.legend()
plt.title("Time series matching with DTW")

plt.tight_layout()
plt.show()

In [ ]:
from extract import *

path = 'exiosuu_LTE_TALAGAKOCAK_GH_BXP_2053_telog'
fe = FileExtract(path)

In [ ]:
from tslearn.metrics import dtw, dtw_path
from tslearn.metrics import lcss, lcss_path
import numpy
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from extract import *

def cal_lcss_path_and_score(s_y1, s_y2):
    path, score = lcss_path(s_y1, s_y2)
    return path, score

def cal_dtw_path_and_score(s_y1, s_y2):
    path, score = dtw_path(s_y1, s_y2)
    return path, score

with open(cf['ENV_'+env]['LOG_STORE_PATH'] + 'GLT_SUKAMULYA_CBN_CM_BXP_2051_telog.log_BXP_2051_radio6626_2022_10_10', "rb") as myfile:
    S = myfile.read()
story_a = json.loads(gzip.decompress(S))

with open(cf['ENV_'+env]['LOG_STORE_PATH'] + 'exiosuu_LTE_TALAGAKOCAK_GH_2052.log_BXP_2052_radio6626_2022_10_10', "rb") as myfile:
    S = myfile.read()
story_b = json.loads(gzip.decompress(S))

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from bisect import bisect

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def del_list_inplace(l, id_to_del):
    for i in sorted(id_to_del, reverse=True):
        del(l[i])

keyword = 'Pma(c)'
final = pd.DataFrame()
kv_a_global_indices = [int(item) for item in story_a['kv']['txlProcBranchH'][keyword][-1]]
kv_b_global_indices = [int(item) for item in story_b['kv']['txlProcBranchH'][keyword][-1]]

kv_a = [[i, kv_a_global_indices[i], float(item)] for i, item in enumerate(story_a['kv']['txlProcBranchH'][keyword][0])]
kv_b = [[i, kv_b_global_indices[i], float(item)] for i, item in enumerate(story_b['kv']['txlProcBranchH'][keyword][0])]

kv_a = pd.DataFrame(kv_a, columns=['x', 'global_index','value'])
kv_a['category'] = 'story_a'
kv_a['loop'] = 'origin'

kv_b = pd.DataFrame(kv_b, columns=['x', 'global_index','value'])
kv_b['category'] = 'story_b'
kv_b['loop'] = 'origin'

final = final.append(kv_a).reset_index(drop=True)
final = final.append(kv_b).reset_index(drop=True)

for loop,_ in enumerate(range(0, 2)):
    path, score = lcss_path(NormalizeData(list(kv_a.value.values)), NormalizeData(list(kv_b.value.values)), eps=0.1)
    o_a = [[i, kv_a['global_index'][item[0]], kv_a['value'][item[0]]] for i, item in enumerate(path)]
    o_b = [[i, kv_b['global_index'][item[1]], kv_b['value'][item[1]]] for i, item in enumerate(path)]
    tmp_a = pd.DataFrame(o_a, columns=['x', 'global_index','value'])
    tmp_a['category'] = 'story_a'
    tmp_a['loop'] = 'loop'+str(loop)
    tmp_b = pd.DataFrame(o_b, columns=['x', 'global_index','value'])
    tmp_b['category'] = 'story_b'
    tmp_b['loop'] = 'loop'+str(loop)
    final = final.append(tmp_a).reset_index(drop=True)
    final = final.append(tmp_b).reset_index(drop=True)

    kv_a = kv_a.drop(kv_a.index[ [item[0] for item in path] ]).reset_index(drop=True)
    kv_b = kv_b.drop(kv_b.index[ [item[1] for item in path] ]).reset_index(drop=True)
    
sns.relplot(
    data=final, kind="line",
    x="x", y="value", col="category", hue="loop")

In [ ]:
tmp = final.loc[(final['loop'] != 'origin'), :].reset_index(drop=True)
sns.relplot(
    data=tmp, kind="line",
    x="x", y="value", col="category", hue="loop")

In [ ]:
sns.relplot(
    data=final, kind="line",
    x="global_index", y="value", col="category", hue="loop")

In [ ]:
process = 'txlProcBranchH'

highlight_story = {}
story_a_error_global_indices = []
for i, p in enumerate(story_a['inverted_index_table']['error']['process']):
    if p == process:
        story_a_error_global_indices.append(int(story_a['inverted_index_table']['error']['x'][i]))
        
story_b_error_global_indices = []
for i, p in enumerate(story_b['inverted_index_table']['error']['process']):
    if p == process:
        story_b_error_global_indices.append(int(story_b['inverted_index_table']['error']['x'][i]))

highlight_story = {'story_a':story_a_error_global_indices, 'story_b':story_b_error_global_indices}

In [ ]:
res = []
for loop in set(final.loop.values):
    for story in set(final.category.values):
        gi = final.loc[(final['loop'] == loop)&(final['category'] == story), :].global_index.values
        print(loop, story, len(gi))
        tmp = []
        for index in highlight_story[story]:
            tmp.append(bisect(gi, index))
        res.append({'loop':loop, 'category':story, 'position': sorted(set(tmp), key=tmp.index)})
res

In [ ]:
np.std(final.loc[(final['loop'] == 'loop1')&(final['category'] == 'story_a'), :].value.values)

In [ ]:
np.std(final.loc[(final['loop'] == 'loop0')&(final['category'] == 'story_a'), :].value.values)

In [ ]:
a = 132
b = [0, 10, 30, 60, 100, 150, 210, 280, 340, 480, 530]
print(bisect(b, a))

In [ ]:
# 1.值范围 2.相似线段形态 3.highlight落在区间内 4.方差大

In [ ]:
import re
import time

str1 = 'BXP_2: [2022-11-18 12:51:26.550407110] (+0.004950450) radio6626 com_ericsson_trithread:INFO: { cpu_id = 2 }, { process = "txlProcBranchI", fileAndLine = "dpdController.cc:1886", msg = "Gain started. Pma:-21.79[-41.54 -9.50] dB, DpdPma:-25.83[-26.49 -25.09] dB, Pmb:-21.79, TorPmb:-21.92[-56.79 -9.50] dB, avgTxPma:-inf dB, pmDpdIrqStat:0x00000000, pmScaleFactor: 65K" }\n'
regex = "\[%{TIMESTAMP:time}\] %{DROP:tmp}txAtt:%{INT:txAtt}, %{DROP:tmp}avgIMpa0:%{INT:avgIMpa0} "
# regex = "%{STRING:device}: \[%{TIMESTAMP:time}\] "
v_regex = regex
for i, r in enumerate(re.findall('%\{.*?\}', regex)):
    regex = regex.replace(r, '(.*?)')
    v_regex = v_regex.replace(r, '<font color="color:#FFFFFF">'+"\\"+str(i+1)+'</font>')

re.findall(regex, str1)

In [ ]:
def word_replace(word):
    return word.group(0).replace(word.group(1), '<font color="color:#FFFFFF">'+word.group(1)+'</font>')
    
special_symbols = ['/','\*','\{','\}','\[','\]','\(','\)','#','+','-','!','=',':',',','"','\'','>','<','@','$','%','^','\&','\|',' ']
reg = '['+'|'.join(special_symbols)+']' +'(txAtt|txlProcBranchH)'+ '['+'|'.join(special_symbols)+']'
re.sub(reg, word_replace,str1)

In [ ]:
# re.sub(regex, r''+r'\1: [\2] \\(\3\\) \4 \5: \6txAtt:\7, txAttPeak:\8,\9torTemperature:\10 ', str1)
re.sub(regex, v_regex, str1).replace('\\','')

In [ ]:
def udpate_value(item, value):
    item['value'] = value
    return item
tmp = [{'a':1, 'value':3},{'a':1, 'value':4}]
list(map(udpate_value, tmp, ['aa', 'aa']))

In [ ]:
import os
import re
import pandas as pd
import gzip

def iterate_files_in_directory(directory):
    # iterate over files in
    # that directory
    res = []
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            res.append(filename)
    return res

# _dir = 'ELOG_READ_FULL'
# for num, gz in enumerate(iterate_files_in_directory(_dir)):
#     print(num, gz)
#     path = _dir+'/'+gz
#     regex = "^(.*?)  +(.*?)  +(.*?)  +(.*?)  +(.*?)  +(.*?)  +(.*?)  +"
#     with gzip.open(path, 'r') as f:
#         lines = [line.decode("utf-8").replace('\n','') for line in f.readlines()]
#         # find device info
#         devices = []
#         for i, line in enumerate(lines):
#             if (len(re.findall(regex, line)) > 0) & ('Radio 6626' in line):
#                 j = i
#                 while ('coli>' not in lines[j]):
#                     if len(re.findall(regex, lines[j])) > 0:
#                         devices.append(re.findall(regex, lines[j])[0])
#                     j = j + 1
#                 break

#         # find log range
#         flag = False
#         res = {}
#         for line in lines:
#             if ('coli>' in line) & ('lhsh' in line) & ('elog read' in line):
#                 flag = True
#                 dev = re.findall('lhsh (.*?) elog read', line)[0]
#                 res[dev] = []

#             if flag:
#                 res[dev].append(line)

#         devices = pd.DataFrame(devices, columns=['LDN','LINKHANDLER','HWTYPE','PRODUCT_ID','REV','SERIAL','PRODUCTION_DATE'])
#         for dev in res.keys():
#             tmp = devices.loc[(devices['LINKHANDLER'] == dev),['LINKHANDLER','HWTYPE','PRODUCT_ID','REV','SERIAL','PRODUCTION_DATE']]
#             if len(tmp) > 0:
#                 f2 = open('save_log/'+path.split('/')[1].split('.')[0]+'-'+'-'.join(tmp.values[0]).replace('/','_'), "w")
#                 f2.write('\n'.join(res[dev]))
#                 f2.close()
#     f.close()
    
_dir = 'YanLin'
for num, gz in enumerate(iterate_files_in_directory(_dir)):
    print(num, gz)
    path = _dir+'/'+gz
    regex = "^(.*?)  +(.*?)  +(.*?)  +(.*?)  +(.*?)  +(.*?)  +(.*?)  +"
    with open(path, 'r') as f:
        lines = [line.replace('\n','') for line in f.readlines()]
        # find device info
        devices = []
        for i, line in enumerate(lines):
            if (len(re.findall(regex, line)) > 0) & ('Radio 6626' in line):
                j = i
                while ('coli>' not in lines[j]):
                    if len(re.findall(regex, lines[j])) > 0:
                        devices.append(re.findall(regex, lines[j])[0])
                    j = j + 1
                break

        # find log range
        flag = False
        res = {}
        for line in lines:
            if ('coli>' in line) & ('lhsh' in line) & ('elog read' in line):
                flag = True
                dev = re.findall('lhsh (.*?) elog read', line)[0]
                res[dev] = []

            if flag:
                res[dev].append(line)

        devices = pd.DataFrame(devices, columns=['LDN','LINKHANDLER','HWTYPE','PRODUCT_ID','REV','SERIAL','PRODUCTION_DATE'])
        for dev in res.keys():
            tmp = devices.loc[(devices['LINKHANDLER'] == dev),['LINKHANDLER','HWTYPE','PRODUCT_ID','REV','SERIAL','PRODUCTION_DATE']]
            if len(tmp) > 0:
                f2 = open('save_log/'+path.split('/')[1].split('.')[0]+'-'+'-'.join(tmp.values[0]).replace('/','_'), "w")
                f2.write('\n'.join(res[dev]))
                f2.close()
    f.close()

In [ ]:
path = _dir+'/LTE_CIBURUY_LEBAK_MT.log.gz'
regex = "^(.*?)  +(.*?)  +(.*?)  +(.*?)  +(.*?)  +(.*?)  +(.*?)  +"
with gzip.open(path, 'r') as f:
    lines = [line.decode("utf-8").replace('\n','') for line in f.readlines()]
    # find device info
    for i, line in enumerate(lines):
        if (len(re.findall(regex, line)) > 0) & ('Radio 6626' in line):
            devices = []
            j = i
            while ('coli>' not in lines[j]):
                print(lines[j])
                if len(re.findall(regex, lines[j])) > 0:
                    devices.append(re.findall(regex, lines[j])[0])
                j = j + 1
            break

    # find log range
    flag = False
    res = {}
    for line in lines:
        if ('coli>' in line) & ('lhsh' in line) & ('elog read' in line):
            flag = True
            dev = re.findall('lhsh (.*?) elog read', line)[0]
            res[dev] = []

        if flag:
            res[dev].append(line)

    devices = pd.DataFrame(devices, columns=['LDN','LINKHANDLER','HWTYPE','PRODUCT ID','REV','SERIAL','PRODUCTION DATE'])
    for dev in res.keys():
        tmp = devices.loc[(devices['LINKHANDLER'] == dev),['LINKHANDLER','HWTYPE','REV','SERIAL']]
        if len(tmp) > 0:
            f2 = open('save_log/'+path.split('/')[1].split('.')[0]+'-'+'-'.join(tmp.values[0]), "w")
            f2.write('\n'.join(res[dev]))
            f2.close()
f.close()

In [ ]:
import os
import re
from file import TextFile

def iterate_files_in_directory(directory):
    # iterate over files in
    # that directory
    res = []
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            res.append(filename)
    return res

final = []

In [ ]:
_dir = 'save_log2'
 
for num, filename in enumerate(iterate_files_in_directory(_dir)):
    print(num, filename)
    fe = TextFile(_dir+'/'+filename)
    header = filename.split('-')
    res = fe.search('test', '', ["\[%{TIMESTAMP:time}\] %{STRING:msg}$"], ['(ABN & raiseFaultAndRollback) < R13C193', 'R13C193 < (ABN & raiseFaultAndRollback)'], [])
    conditions = []
    condition1_happen_time = []
    condition2_happen_time = []

    # condition 1
    flag = True
    for c in fe.searchs[res].res_condition['(ABN & raiseFaultAndRollback) < R13C193']:
        if (c[2] < c[1]):
            if (re.findall('\[(.*?)\] ', fe.lines[c[2]])[0] >= '221100 000000') | (re.findall('\[(.*?)\] ', fe.lines[c[3]])[0] >= '221100 000000'):
                condition1_happen_time.append(re.findall('\[(.*?)\] ', fe.lines[c[2]])[0])
                flag = False
    if flag:
        conditions.append(False)
    else:
        conditions.append(True)

    # condition 2
    flag = True
    for c in fe.searchs[res].res_condition['R13C193 < (ABN & raiseFaultAndRollback)']:
        if (c[0] < c[3]):
            if (re.findall('\[(.*?)\] ', fe.lines[c[2]])[0] >= '221100 000000') | (re.findall('\[(.*?)\] ', fe.lines[c[3]])[0] >= '221100 000000'):
                condition2_happen_time.append(re.findall('\[(.*?)\] ', fe.lines[c[3]])[0])
                flag = False
    if flag:
        conditions.append(False)
    else:
        conditions.append(True)


    header.extend(conditions)
    header.append('||'.join(list(set(condition1_happen_time))))
    header.append('||'.join(list(set(condition2_happen_time))))
    final.append(header)

In [ ]:
final = []
_dir = 'save_log2'
filename = 'ULTE_MATOA-BXP_5-Radio 6626 66B1 66B3 C-KRC 161 924_1-R2B-EA8B224402-20220831'
fe = TextFile(_dir+'/'+filename)
header = filename.split('-')
res = fe.search('test', '', ["\[%{TIMESTAMP:time}\] %{STRING:msg}$"], ['(ABN & raiseFaultAndRollback) < R13C193', 'R13C193 < (ABN & raiseFaultAndRollback)'], [])
conditions = []
condition1_happen_time = []
condition2_happen_time = []

# condition 1
flag = True
for c in fe.searchs[res].res_condition['(ABN & raiseFaultAndRollback) < R13C193']:
    if (c[2] < c[1]):
        if (re.findall('\[(.*?)\] ', fe.lines[c[2]])[0] >= '221100 000000') | (re.findall('\[(.*?)\] ', fe.lines[c[3]])[0] >= '221100 000000'):
            condition1_happen_time.append(re.findall('\[(.*?)\] ', fe.lines[c[2]])[0])
            flag = False
if flag:
    conditions.append(False)
else:
    conditions.append(True)
    
# condition 2
flag = True
for c in fe.searchs[res].res_condition['R13C193 < (ABN & raiseFaultAndRollback)']:
    if (c[0] < c[3]):
        if (re.findall('\[(.*?)\] ', fe.lines[c[2]])[0] >= '221100 000000') | (re.findall('\[(.*?)\] ', fe.lines[c[3]])[0] >= '221100 000000'):
            condition2_happen_time.append(re.findall('\[(.*?)\] ', fe.lines[c[3]])[0])
            flag = False
if flag:
    conditions.append(False)
else:
    conditions.append(True)
        
        
header.extend(conditions)
header.append('||'.join(list(set(condition1_happen_time))))
header.append('||'.join(list(set(condition2_happen_time))))
final.append(header)
final = pd.DataFrame(final, columns=['SiteName', 'LINKHANDLER', 'HWTYPE', 'PRODUCT_ID', 'REV', 'SERIAL', 'PRODUCTION_DATE', '(ABN & raiseFaultAndRollback) < R13C193', 'R13C193 < (ABN & raiseFaultAndRollback)', 'Before Update Lin Fault Date', 'After Update Lin Fault Date'])

In [ ]:
import re
import time

# str1 = 'BXP_3: [2022-11-23 22:04:44.572174570] (+0.000333470) radio6626 com_ericsson_trithread:INFO: { cpu_id = 1 }, { process = "txlProcBranchB", fileAndLine = "dpdController.cc:1887", msg = "txAtt:289, txAttPeak:0, dpGainLoopEnable:true, dpGainCtrlType:DSA_AD_TXFE, torTemperature:495 (0.1C), torGainBackoff:0 (0.01dB), torGainLin:3.49543(0.01dB), torStepBit:6, cc0Ctrl1=0x00000118 , avgIMpa0:1980 [mAmp]" }'
str1 = "BXP_2: [221120 164014] 27: PA measured values for driver name: DpaVddSv:7; value: 26992; branch Id: 7"
# regex = "\[%{TIMESTAMP:time}\] \(%{STRING:cost}\) "
# regex = "\[%{TIMESTAMP:time}\] %{DROP:tmp}Pma:%{FLOAT:Pma0}\[%{DROP:tmp1}DpdPma:%{FLOAT:DpdPma0}\[%{DROP:tmp2}Pmb:%{FLOAT:Pmb}, TorPmb:%{FLOAT:TorPmb0}\[%{FLOAT:TorPmb1} %{FLOAT:TorPmb2}\] "
# regex = "\[%{TIMESTAMP:time}\] %{DROP:tmp}txAtt:%{FLOAT:txAtt}, %{DROP:tmp1}torTemperature:%{INT:torTemperature} %{DROP:tmp2}avgIMpa0:%{INT:avgIMpa0} "
regex = "\[%{TIMESTAMP:time}\] %{DROP:tmp}DpaVddSv:7; value: %{INT:DpaVddSv7}; "
v_regex = regex
for i, r in enumerate(re.findall('%\{.*?\}', regex)):
    regex = regex.replace(r, '(.*?)')
    v_regex = v_regex.replace(r, '<font color="color:#FFFFFF">'+"\\"+str(i+1)+'</font>')

re.findall(regex, str1)

In [ ]:
time1 = time.time()

is_linearization_fault = fe.search('is_linearization_fault', 'txlProcBranchI & linearization & fault', 
                ["%{STRING:device}: \[%{TIMESTAMP:time}\] \(%{STRING:cost}\) %{STRING:name} %{STRING:trace}: %{DROP:tmp}txAtt:%{INT:txAtt}, txAttPeak:%{INT:txAttPeak},%{DROP:tmp}torTemperature:%{INT:torTemperature} ",
                "%{STRING:device}: \[%{TIMESTAMP:time}\] \(%{STRING:cost}\) %{STRING:name} %{STRING:trace}: %{DROP:tmp}Pma:%{FLOAT:Pma0}\[%{FLOAT:Pma1} %{FLOAT:Pma2}\] dB, ",
                "%{STRING:device}: \[%{TIMESTAMP:time}\] "
                ], [], [])
print(time.time() - time1)

In [ ]:
import uuid
import json

path = 'save_log/Visby_Telog_All_Branch_txAtt_torpmb_avgImpa.txt'
with open(path, 'r') as f:
    theme = json.loads(f.read())

In [ ]:
branch = ['A','B','C','D','E','F','G','H','I','J','K','L']
for index, item in enumerate(theme['search']):
    if index < 12:
        item['desc'] = 'Branch' + branch[index] + '_Common_Use_KeyWords'
        item['search'] = 'txlProcBranchA & (txAtt | pmb) | ((txlProcBranchA  | TxBranchCtrlA) & (fault | error | abn))'.replace('BranchA', 'Branch'+branch[index]).replace('CtrlA', 'Ctrl'+branch[index])
        item['regexs'] = ["\[%{TIMESTAMP:time}\] %{DROP:tmp}Pma:%{FLOAT:Pma0}\[%{DROP:tmp1}DpdPma:%{FLOAT:DpdPma0}\[%{DROP:tmp2}Pmb:%{FLOAT:Pmb}, TorPmb:%{FLOAT:TorPmb0}\[%{FLOAT:TorPmb1} %{FLOAT:TorPmb2}\] ",
                            "\[%{TIMESTAMP:time}\] %{DROP:tmp}txAtt:%{FLOAT:txAtt}, %{DROP:tmp1}torTemperature:%{INT:torTemperature} %{DROP:tmp2}avgIMpa0:%{INT:avgIMpa0} ",
                            "\[%{TIMESTAMP:time}\] \(%{DROP:tmp}\) "
                            ]
        item['highlights'] = [["fault,error","#fa0000"],["abn","#fbff00"]]
theme['keyValueTree'] = {}

In [8]:
import uuid

branch = ['A','B','C','D','E','F','G','H','I','J','K','L']
config = {'search':[], 'keyValueTree':[]}
for index in branch:
    item = {}
    item['uid'] = str(uuid.uuid4()).replace('-','')
    item['desc'] = 'Branch' + str(index) + '_Common_Use_KeyWords'
    item['search'] = 'txlProcBranch'+str(index)+' & (txAtt | pmb) | ((txlProcBranch'+str(index)+' | TxBranchCtrl'+str(index)+') & linearization & fault)'
    item['regexs'] = ["\[%{TIMESTAMP:time}\] %{DROP:tmp}Pma:%{FLOAT:Pma0}\[%{DROP:tmp1}DpdPma:%{FLOAT:DpdPma0}\[%{DROP:tmp2}Pmb:%{FLOAT:Pmb}, TorPmb:%{FLOAT:TorPmb0}\[%{FLOAT:TorPmb1} %{FLOAT:TorPmb2}\] ",
                        "\[%{TIMESTAMP:time}\] %{DROP:tmp}txAtt:%{FLOAT:txAtt}, %{DROP:tmp1}torTemperature:%{INT:torTemperature} %{DROP:tmp2}avgIMpa0:%{INT:avgIMpa0} ",
                        "\[%{TIMESTAMP:time}\] \(%{DROP:tmp}\) "
                        ]
    item['highlights'] = [["fault","#fa0000"]]
    config['search'].append(item)

In [ ]:
import uuid
branch = ['A','B','C','D','E','F','G','H','I','J','K','L']
config = {'search':[], 'keyValueTree':[]}
for index in range(12):
    item = {}
    item['uid'] = str(uuid.uuid4()).replace('-','')
    item['desc'] = 'PA measured values for driver name ' + 'Branch' + str(branch[index])
    item['search'] = 'PA & measured & driver'
    item['regexs'] = [
                        "\[%{TIMESTAMP:time}\] %{DROP:tmp}PaVddSv:"+str(index)+"; value: %{INT:PaVddSv"+str(index)+"}; ",
                        "\[%{TIMESTAMP:time}\] %{DROP:tmp}DpaVddSv:"+str(index)+"; value: %{INT:DpaVddSv"+str(index)+"}; ",
                        "\[%{TIMESTAMP:time}\] %{DROP:tmp}IMpaSv:"+str(index)+"\.0; value: %{INT:IMpaSv"+str(index)+"0}; ",
                        "\[%{TIMESTAMP:time}\] %{DROP:tmp}IMpaSv:"+str(index)+"\.1; value: %{INT:IMpaSv"+str(index)+"1}; ",
                        "\[%{TIMESTAMP:time}\] %{DROP:tmp}IDpaSv:"+str(index)+"\.0; value: %{INT:IDpaSv"+str(index)+"0}; ",
                        "\[%{TIMESTAMP:time}\] %{DROP:tmp}IDpaSv:"+str(index)+"\.1; value: %{INT:IDpaSv"+str(index)+"1}; ",
                        "\[%{TIMESTAMP:time}\] \(%{DROP:tmp}\) "
                        ]
    item['highlights'] = []
    config['search'].append(item)

In [ ]:
import uuid
branch = ['A','B','C','D','E','F','G','H','I','J','K','L']
config = {'search':[], 'keyValueTree':[]}
for index in branch:
    item = {}
    item['uid'] = str(uuid.uuid4()).replace('-','')
    item['desc'] = 'Recieve IRQ event & powerFailureHandler ' + 'Branch' + index
    item['search'] = '(txlProcBranch'+index+' & txAtt) | (Recieve & IRQ & event) | powerFailureHandler'
    item['regexs'] = [
                        "\[%{TIMESTAMP:time}\] %{DROP:tmp}txAtt:%{FLOAT:txAtt}, %{DROP:tmp1}torTemperature:%{INT:torTemperature} %{DROP:tmp2}avgIMpa0:%{INT:avgIMpa0} ",
                        "\[%{TIMESTAMP:time}\] \(%{DROP:tmp}\) "
                        ]
    item['highlights'] = [["IRQ","#fa0000"],["powerFailureHandler","#fbff00"]]
    config['search'].append(item)

In [9]:
with open('D:\\projects\\ericsson_flow\\themes\\Visby_Telog_YanLin_exception_type.txt', 'w') as f:
    f.write(json.dumps(config))

In [ ]:
import time
import numpy as np
import tracemalloc
from multiprocessing import Process, Manager

In [ ]:
tracemalloc.start()
start_time = time.time()
path = 'themes/AIR_teread.log'
# path = 'themes/LE_JLSANTANG_PL_20221125_054403_Logfiles/LE_JLSANTANG_PL.log'
# path = 'save_log/10.19.212.78.log'
with open(path, 'r') as f:
#     np.char.decode(f.readlines().astype(np.bytes_), 'UTF-8')
#     data = np.array(f.readlines(), dtype='S600')
    L = Manager().list(f.readlines())
#     lines = np.array([line[:600] for line in f.readlines()])
print(time.time() - start_time)
current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage {current/1e6}MB; Peak: {peak/1e6}MB")

In [ ]:
import json
from file import TextFile

config = 'D:\\projects\\ericsson_flow\\themes\\Visby_Elog_Common_Use_KeyValue.txt'
file = 'D:\\projects\\ericsson_flow\\themes\\LE_JLSANTANG_PL_dcg_e2.log'
with open(config, 'r') as f:
    config = json.loads(f.read())
    
fe = TextFile('', file, 'test', 'single')
search = config['search'][0]
uid = fe.search('', search['desc'], search['search'], search['regexs'], [], search['highlights'])

In [ ]:
import io
import re
import gzip
import zipfile

def analysis_telog(lines, filters):
    data = {}
    flag = False
    satisfies = [False for _ in filters]
    device = ''
    for line in lines:
        line = line.decode("utf-8")
        if ('te log read' in line) & ('lhsh' in line):
            if device != '':
                if False in satisfies:
                    data[device] = []
                    del data[device]

            flag = True
            device = re.findall('lhsh (.*?)te log read', line)[0].strip()
            data[device] = []
            satisfies = [False for _ in filters]
                
        if flag:
            for index, f in enumerate(filters):
                if f in line:
                    satisfies[index] = True
            data[device].append(line)
    if device != '':
        if False in satisfies:
            data[device] = []
            del data[device]
    return data

def analysis_elog(lines, filters):
    data = {}
    flag = False
    satisfies = [False for _ in filters]
    device = ''
    for line in lines:
        line = line.decode("utf-8")
        if ('dcg run' in line) & ('lhsh' in line):
            if device != '':
                if False in satisfies:
                    data[device] = []
                    del data[device]
                    
            flag = True
            device = re.findall('lhsh (.*?)dcg run', line)[0].strip()
            data[device] = []
            satisfies = [False for _ in filters]
                
        if flag:
            for index, f in enumerate(filters):
                if f in line:
                    satisfies[index] = True
            data[device].append(line)
    if device != '':
        if False in satisfies:
            data[device] = []
            del data[device]
    return data
    

dcgm_path = 'D:\\projects\\ericsson_flow\\new_dcgm\\LE_JLSANTANG_PL_221128_192943_WIB_MSRBS-L_CXP9024418-15_R53K11_dcgm.zip'
save_path = 'D:\\projects\\ericsson_flow\\new_files\\'
telog_name = 'teread.log'
telog_filters = ['radio6626']
elog_filters = ['HW fault']
with zipfile.ZipFile(dcgm_path, 'r') as outer:
    telog = ''
    elog = ''
    for file in outer.filelist:
        if '_logfiles.zip' in file.filename:
            telog = file.filename
        if '_dcg_e2.log.gz' in file.filename:
            elog = file.filename
            
    with outer.open(telog, 'r') as nest:
        zfile = io.BytesIO(nest.read())
        with zipfile.ZipFile(zfile) as nested_zip:
            with nested_zip.open(telog_name, 'r') as file:
                telog_data = analysis_telog(file.readlines(), telog_filters)
            
    with outer.open(elog, 'r') as nest:
        zfile = io.BytesIO(nest.read())
        lines = gzip.open(zfile, 'r')
        elog_data = analysis_elog(lines, elog_filters)
        

for dev in set(telog_data.keys()).intersection(set(elog_data.keys())):
    with open(save_path+telog.replace('_logfiles.zip', '')+'_'+dev+'_telog.log', 'w', encoding='utf-8') as f:
        f.write(''.join(telog_data[dev]))
        
    with open(save_path+elog.replace('_dcg_e2.log.gz', '')+'_'+dev+'_elog.log', 'w', encoding='utf-8') as f:
        f.write(''.join(elog_data[dev]))

In [4]:
from special import *

dcgm_dir_path = 'D:\\projects\\ericsson_flow\\new_dcgm\\'
save_path = 'D:\\projects\\ericsson_flow\\new_files\\'

telog_filters = ['radio6626']
elog_filters = ['HW fault']
take_apart_dcgm(dcgm_dir_path, save_path, telog_filters, elog_filters)

0 GLTE_PONDOK_JATI_TGX_PL_221207_072225_WIB_MSRBS-GL_CXP2010174-1_R50K10_dcgm.zip
1 LE_KAMPUSC_UBAYA_ST_221207_075025_WIB_MSRBS-L_CXP9024418-15_R53K11_dcgm.zip
2 LE_KEDOYARAYA_221207_072632_WIB_MSRBS-LW_CXP9024418-15_R53K11_dcgm.zip
3 LTE_FLAMBOYAN_TIGA_TB_221207_073131_WIB_MSRBS-LW_CXP9024418-15_R53K11_dcgm.zip
4 LTE_HABDULLAH_KLPDUA_EP_221207_072410_WIB_MSRBS-L_CXP9024418-15_R53K11_dcgm.zip
